In [2]:
!pip install torch
!pip install transformers


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 7.9/7.9 MB 527.4 kB/s eta 0:00:00
     -------------------------------------- 311.7/311.7 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 1.8 MB/s eta 0:00:00
     ------------------------------------ 277.2/277.2 kB 950.0 kB/s eta 0:00:00
     -------------------------------------- 166.4/166.4 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd

# Load your dataset
df = pd.read_csv("BERT Dataset.csv")  # Replace with your dataset path

# Display the first few rows to understand the structure
print(df.head())


                                           Sentences  \
0  excellent service....very professional electri...   
1                                  nice and good app   
2          came on time work proeffetionaly complete   
3                first time user and very satisfied.   
4          nice facilities and rates is to much high   

                                    Broken Sentences       OpinionTerm1  \
0  ['excellent service', '.very professional elec...  excellent service   
1                              ['nice and good app']           good app   
2   ['came on time', 'work proeffetionaly complete']            on time   
3            ['first time user ', ' very satisfied']    first time user   
4     ['nice facilities ', ' rates is to much high']    nice facilities   

  Predicted Opinion Term 1   Aspect Category1 AspectPolarity1   \
0        excellent service    Company_Service         positive   
1                 good app        App_Utility         positive   
2             

In [5]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [9]:
from transformers import BertTokenizer, BertForTokenClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm

# Tokenize the input data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Prepare input data
def prepare_data_opinion_term(df, tokenizer, max_len=128):
    input_ids = []
    attention_masks = []

    for text in tqdm(df['Sentences']):
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens=True,
                            max_length=max_len,
                            padding='max_length',
                            return_tensors='pt',
                            truncation=True
                    )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return TensorDataset(input_ids, attention_masks)

train_data_opinion = prepare_data_opinion_term(train_df, tokenizer)
test_data_opinion = prepare_data_opinion_term(test_df, tokenizer)

# Fine-tune BERT for opinion term prediction
model_opinion = BertForTokenClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2  # Change based on your specific task
)

# Set up data loaders
batch_size = 8
train_loader_opinion = DataLoader(train_data_opinion, batch_size=batch_size, shuffle=True)
test_loader_opinion = DataLoader(test_data_opinion, batch_size=batch_size, shuffle=False)

# Set up optimizer and training parameters
optimizer_opinion = AdamW(model_opinion.parameters(), lr=2e-5)

# Fine-tune the model
epochs = 3
for epoch in range(epochs):
    model_opinion.train()
    for batch in tqdm(train_loader_opinion):
        optimizer_opinion.zero_grad()
        input_ids, attention_masks = batch
        outputs = model_opinion(input_ids, attention_mask=attention_masks)
        # Add your custom loss function here
        loss = outputs.loss
        loss.backward()
        optimizer_opinion.step()

# Evaluate the model on the test set
model_opinion.eval()
# Add your evaluation code here


100%|██████████████████████████████████████████████████████████████████████████████| 833/833 [00:00<00:00, 3062.48it/s]


ConnectionError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/tf_model.h5 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000016B7BDDACD0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))